In [5]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# -------------------------------
# Step 1: Define raw URLs for train, test, and prod datasets.
# -------------------------------
train_url = "https://raw.githubusercontent.com/Jupudy-Nagendra/MLOPS/main/Dataset/Parquet/Metro_Interstate_Traffic_Volume_train.parquet"
test_url  = "https://raw.githubusercontent.com/Jupudy-Nagendra/MLOPS/main/Dataset/Parquet/Metro_Interstate_Traffic_Volume_test.parquet"
prod_url  = "https://raw.githubusercontent.com/Jupudy-Nagendra/MLOPS/main/Dataset/Parquet/Metro_Interstate_Traffic_Volume_prod.parquet"

# Helper function to load a parquet file from GitHub.
def load_parquet_from_url(url):
    response = requests.get(url)
    response.raise_for_status()
    return pd.read_parquet(BytesIO(response.content))

# Load the train dataset (for example).
train_df = load_parquet_from_url(train_url)
test_df  = load_parquet_from_url(test_url)
prod_df  = load_parquet_from_url(prod_url)

# -------------------------------
# Step 2: Define a custom transformer to extract datetime features
# -------------------------------
class DateTimeExtractor(BaseEstimator, TransformerMixin):
    """
    Transformer that converts the 'date_time' column into datetime format
    and extracts new columns: year, month, day, and hour.
    Optionally drops the original date_time column.
    """
    def __init__(self, column="date_time", drop_original=True):
        self.column = column
        self.drop_original = drop_original
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        # Convert the column to datetime
        X[self.column] = pd.to_datetime(X[self.column])
        # Extract datetime components
        X['year'] = X[self.column].dt.year
        X['month'] = X[self.column].dt.month
        X['day'] = X[self.column].dt.day
        X['hour'] = X[self.column].dt.hour
        # Optionally drop the original date_time column
        if self.drop_original:
            X = X.drop(columns=[self.column])
        return X

# Build the datetime extraction pipeline.
datetime_pipeline = Pipeline(steps=[
    ('datetime_extractor', DateTimeExtractor(column="date_time", drop_original=True))
])

# -------------------------------
# Step 3: Define a custom holiday transformer
# -------------------------------
class HolidayBinaryTransformer(BaseEstimator, TransformerMixin):
    """
    Transformer that converts the 'holiday' column into a binary feature:
      - 0 if the value is missing or equals "None"
      - 1 otherwise.
    Implements get_feature_names_out for compatibility.
    """
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Expect X as a DataFrame with column 'holiday'
        binary = ((~pd.isnull(X)) & (X != "None")).astype(int)
        # Ensure a 2D array output
        if isinstance(binary, pd.Series):
            binary = binary.to_frame()
        return binary.values

    def get_feature_names_out(self, input_features=None):
        return np.array(['holiday_binary'])

# Instantiate our custom holiday transformer.
holiday_transformer = HolidayBinaryTransformer()

# -------------------------------
# Step 4: Define pipelines for numeric and categorical features
# -------------------------------
# After datetime extraction, the DataFrame will include:
# Numeric columns: 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'year', 'month', 'day', 'hour'
# Categorical columns (excluding 'holiday'): 'weather_main', 'weather_description'
numeric_cols = ['temp', 'rain_1h', 'snow_1h', 'clouds_all', 'year', 'month', 'day', 'hour']
categorical_cols = ['weather_main', 'weather_description']

# Numeric pipeline: impute missing values and scale.
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical pipeline: impute missing values and one-hot encode.
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# -------------------------------
# Step 5: Create a ColumnTransformer to combine all preprocessing steps
# -------------------------------
# The preprocessor applies:
# - Numeric processing on numeric_cols.
# - Categorical processing on categorical_cols.
# - The holiday transformer on the 'holiday' column.
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_pipeline, numeric_cols),
    ('cat', categorical_pipeline, categorical_cols),
    ('holiday', holiday_transformer, ['holiday'])
])

# -------------------------------
# Step 6: Build the full pipeline
# -------------------------------
# Chain datetime extraction and the preprocessor.
full_pipeline = Pipeline(steps=[
    ('datetime', datetime_pipeline),   # Extract datetime features
    ('preprocessor', preprocessor)       # Process numeric, categorical, and holiday columns
    # A model step can be added here, e.g., ('model', SomeRegressor())
])

# -------------------------------
# Step 7: Prepare feature data and transform
# -------------------------------
# Remove the target variable ('traffic_volume') from the train dataset.
X_train_features = train_df.drop(columns=['traffic_volume'])

# Fit and transform the features using the full pipeline.
X_transformed = full_pipeline.fit_transform(X_train_features)

# If OneHotEncoder returns a sparse matrix, convert it to a dense array.
X_transformed_dense = X_transformed if isinstance(X_transformed, np.ndarray) else X_transformed.toarray()

# -------------------------------
# Step 8: Retrieve Output Feature Names
# -------------------------------
# Attempt to get feature names from the preprocessor step.
try:
    feature_names = full_pipeline.named_steps['preprocessor'].get_feature_names_out()
    print("Output Feature Names:")
    print(feature_names)
except Exception as e:
    print("Error retrieving feature names:", e)

# -------------------------------
# Step 9: Inspect Transformed Data
# -------------------------------
print("\nTransformed data shape:", X_transformed_dense.shape)
print("First row of transformed data:")
print(X_transformed_dense[0])


Output Feature Names:
['num__temp' 'num__rain_1h' 'num__snow_1h' 'num__clouds_all' 'num__year'
 'num__month' 'num__day' 'num__hour' 'cat__weather_main_Clear'
 'cat__weather_main_Clouds' 'cat__weather_main_Drizzle'
 'cat__weather_main_Fog' 'cat__weather_main_Haze' 'cat__weather_main_Mist'
 'cat__weather_main_Rain' 'cat__weather_main_Smoke'
 'cat__weather_main_Snow' 'cat__weather_main_Squall'
 'cat__weather_main_Thunderstorm' 'cat__weather_description_SQUALLS'
 'cat__weather_description_Sky is Clear'
 'cat__weather_description_broken clouds'
 'cat__weather_description_drizzle' 'cat__weather_description_few clouds'
 'cat__weather_description_fog' 'cat__weather_description_freezing rain'
 'cat__weather_description_haze'
 'cat__weather_description_heavy intensity drizzle'
 'cat__weather_description_heavy intensity rain'
 'cat__weather_description_heavy snow'
 'cat__weather_description_light intensity drizzle'
 'cat__weather_description_light intensity shower rain'
 'cat__weather_description

In [15]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
import xgboost as xgb

In [11]:
target_col = 'traffic_volume'

# For train data
X_train = train_df.drop(columns=[target_col])
y_train = train_df[target_col]

# For test data
X_test = test_df.drop(columns=[target_col])
y_test = test_df[target_col]

# For production data (no target provided, for inference)
X_prod = prod_df.copy()

# ---------------------------------------
# 4. Transform features using the full pipeline
# ---------------------------------------
X_train_transformed = full_pipeline.fit_transform(X_train)
X_test_transformed  = full_pipeline.transform(X_test)
X_prod_transformed  = full_pipeline.transform(X_prod)

# ---------------------------------------
# 5. Train an XGBoost Regressor
# ---------------------------------------
# Create and train an XGBoost regressor on the preprocessed train data.
model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.01, random_state=42)
model.fit(X_train_transformed, y_train)

# ---------------------------------------
# 6. Evaluate the Model on Test Data
# ---------------------------------------
y_pred_test = model.predict(X_test_transformed)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)
print("Test MAPE: {:.2%}".format(mape_test))

# ---------------------------------------
# 7. Predict on Production Data
# ---------------------------------------
prod_predictions = model.predict(X_prod_transformed)
print("Production predictions (first 5):")
print(prod_predictions[:5])


C:\Users\india\anaconda3\envs\mlops\lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
C:\Users\india\anaconda3\envs\mlops\lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Test MAPE: 107.65%
Production predictions (first 5):
[4740.4897   439.21417  685.2893   568.4065  4834.418  ]


In [22]:
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline

# Create the full pipeline by combining the preprocessor and the XGBoost regressor
xgb_pipeline = Pipeline(steps=[('datetime', datetime_pipeline),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(n_estimators=1000, learning_rate=0.01, random_state=42))
])

# Fit the pipeline on the training data (X_train are the features and y_train is the target)
xgb_pipeline.fit(X_train, y_train)



Pipeline(steps=[('datetime',
                 Pipeline(steps=[('datetime_extractor', DateTimeExtractor())])),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'rain_1h', 'snow_1h',
                                                   'clouds_all', 'year',
                                                   'month', 'day', 'hour']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImp...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.01,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=1000, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [17]:
# Compute error metrics.
mae   = mean_absolute_error(y_test, y_pred_test)
mse   = mean_squared_error(y_test, y_pred_test)
rmse  = np.sqrt(mse)
mape  = mean_absolute_percentage_error(y_test, y_pred_test)
r2    = r2_score(y_test, y_pred_test)

print("Error Metrics on Test Data:")
print("Mean Absolute Error (MAE): {:.2f}".format(mae))
print("Mean Squared Error (MSE): {:.2f}".format(mse))
print("Root Mean Squared Error (RMSE): {:.2f}".format(rmse))
print("Mean Absolute Percentage Error (MAPE): {:.2%}".format(mape))
print("R² Score: {:.2f}".format(r2))

Error Metrics on Test Data:
Mean Absolute Error (MAE): 588.20
Mean Squared Error (MSE): 766041.31
Root Mean Squared Error (RMSE): 875.24
Mean Absolute Percentage Error (MAPE): 107.65%
R² Score: 0.81


In [28]:
!pip install mlflow


  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached waitress-3.0.2-py3-none-any.whl.metadata (5.8 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached protobuf-5.29.3-cp39-cp39-win_amd64.whl.metadata (592 bytes)
  Using cached sqlparse-0.5.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached Mako-1.3.9-py3-none-any.whl.metadata (2.9 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached

In [30]:
import mlflow
import mlflow.sklearn

In [67]:
import os
print("Current working directory:", os.getcwd())

Current working directory: C:\Users\india\Desktop\Jio_Institute\MLOps\Project\Nagendra\MLOPS\Notebooks


In [92]:
%cd C:\Users\india\Desktop\Jio_Institute\MLOps\Project\Nagendra\MLOPS

C:\Users\india\Desktop\Jio_Institute\MLOps\Project\Nagendra\MLOPS


In [94]:
ls

 Volume in drive C is Windows
 Volume Serial Number is E054-376B

 Directory of C:\Users\india\Desktop\Jio_Institute\MLOps\Project\Nagendra\MLOPS

16-03-2025  21:19    <DIR>          .
09-03-2025  18:17    <DIR>          ..
03-03-2025  17:27             3,586 .gitignore
03-03-2025  17:46    <DIR>          .ipynb_checkpoints
03-03-2025  16:38    <DIR>          Dataset
16-03-2025  15:52         2,218,975 Metro_Interstate_Traffic_Volume_Profile.html
16-03-2025  21:19    <DIR>          mlruns
16-03-2025  22:09    <DIR>          Notebooks
03-03-2025  17:27                 7 README.md
               3 File(s)      2,222,568 bytes
               6 Dir(s)  138,483,531,776 bytes free


In [72]:
# Set up MLflow experiment (all runs will be grouped under this experiment)
mlflow.set_experiment("Metro_Interstate_Traffic_Volume")

2025/03/16 21:19:28 INFO mlflow.tracking.fluent: Experiment with name 'Metro_Interstate_Traffic_Volume' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/india/Desktop/Jio_Institute/MLOps/Project/Nagendra/MLOPS/mlruns/411352979973315566', creation_time=1742140168359, experiment_id='411352979973315566', last_update_time=1742140168359, lifecycle_stage='active', name='Metro_Interstate_Traffic_Volume', tags={}>

In [74]:
import mlflow

print("Current Tracking URI:", mlflow.get_tracking_uri())

Current Tracking URI: file:///C:/Users/india/Desktop/Jio_Institute/MLOps/Project/Nagendra/MLOPS/mlruns


In [76]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import mlflow
import mlflow.sklearn
import warnings
warnings.filterwarnings("ignore")

In [78]:
regression_models = {
    "LinearRegression_default": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "XGBoost": XGBRegressor(n_estimators=1000, learning_rate=0.01, random_state=42),
    "ElasticNet": ElasticNet(alpha=0.1, l1_ratio=0.5),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=10, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100, random_state=42),

}

In [80]:
for model_name, reg_model in regression_models.items():
    with mlflow.start_run(run_name=model_name):
        # Log the model name as a parameter.
        mlflow.log_param("model_name", model_name)
        
        # Create a full pipeline: preprocessing + regressor.
        pipeline = Pipeline(steps=[('datetime', datetime_pipeline),
            ('preprocessor', preprocessor),
            ('regressor', reg_model)
        ])
        # Define 5-fold cross-validation.
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)

        # Perform 5-fold cross-validation using negative MAE (we will take the absolute value).
        cv_scores = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')
        mean_cv = np.mean(np.abs(cv_scores))
        std_cv = np.std(np.abs(cv_scores))
        
        # Log cross-validation metrics.
        mlflow.log_metric("cv_mean_MAE", mean_cv)
        mlflow.log_metric("cv_std_MAE", std_cv)
        
        # Train the pipeline on the full training data.
        pipeline.fit(X_train, y_train)
        
        # Evaluate on the test set.
        y_pred = pipeline.predict(X_test)
        mae  = mean_absolute_error(y_test, y_pred)
        mse  = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        r2   = r2_score(y_test, y_pred)
        
        # Log test metrics.
        mlflow.log_metric("test_MAE", mae)
        mlflow.log_metric("test_MSE", mse)
        mlflow.log_metric("test_RMSE", rmse)
        mlflow.log_metric("test_MAPE", mape)
        mlflow.log_metric("test_R2", r2)
        
        # Log hyperparameters if applicable.
        if model_name.startswith("LinearRegression") or model_name.startswith("Ridge") or model_name.startswith("Lasso") or model_name.startswith("ElasticNet"):
            # For these models, log the regularization strength if available.
            if hasattr(reg_model, "alpha"):
                mlflow.log_param("alpha", reg_model.alpha)
            if hasattr(reg_model, "l1_ratio"):
                mlflow.log_param("l1_ratio", reg_model.l1_ratio)
        
        # Log the trained model pipeline.
        mlflow.sklearn.log_model(pipeline, "model")
        
        print(f"{model_name} -> CV MAE: {mean_cv:.4f} ± {std_cv:.4f}, Test MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

2025/03/16 21:19:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


LinearRegression_default -> CV MAE: 1614.5052 ± 11.1028, Test MAE: 1667.9481, RMSE: 5039.5124, R2: -5.3741


2025/03/16 21:19:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Ridge -> CV MAE: 1614.4561 ± 11.1174, Test MAE: 1668.1736, RMSE: 5077.6771, R2: -5.4710


2025/03/16 21:19:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

DecisionTree -> CV MAE: 660.1241 ± 9.5400, Test MAE: 614.6719, RMSE: 1114.6020, R2: 0.6882


2025/03/16 21:21:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForest -> CV MAE: 571.4289 ± 10.7549, Test MAE: 557.6689, RMSE: 876.9476, R2: 0.8070


2025/03/16 21:22:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


GradientBoosting -> CV MAE: 628.0949 ± 3.5558, Test MAE: 631.9079, RMSE: 921.5203, R2: 0.7869


In [1]:
ls

 Volume in drive C is Windows
 Volume Serial Number is E054-376B

 Directory of C:\Users\india\Desktop\Jio_Institute\MLOps\Project\Nagendra\MLOPS\Notebooks

17-03-2025  00:23    <DIR>          .
16-03-2025  21:19    <DIR>          ..
17-03-2025  00:10    <DIR>          .ipynb_checkpoints
16-03-2025  22:18    <DIR>          __pycache__
17-03-2025  00:23            12,855 Notebook.ipynb
17-03-2025  00:09             5,890 Notebook.py
16-03-2025  16:21            58,682 Untitled.ipynb
16-03-2025  22:14            83,605 Untitled1.ipynb
16-03-2025  22:08             3,429 Untitled2.ipynb
16-03-2025  22:17             1,382 Untitled2.py
16-03-2025  22:40            14,417 Untitled3.ipynb
16-03-2025  22:38             3,166 Untitled3.py
16-03-2025  22:02                72 Untitled4.ipynb
               9 File(s)        183,498 bytes
               4 Dir(s)  137,595,740,160 bytes free


In [3]:
!jupyter nbconvert --to script Untitled1.ipynb

[NbConvertApp] Converting notebook Untitled1.ipynb to script
[NbConvertApp] Writing 13962 bytes to Untitled1.py
